In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])


school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [2]:
#Calculate the total number of schools
schools = school_data_complete.school_name.nunique()

#Calculate the total number of students
students = school_data_complete.student_name.count()

#Calculate the total budget
budget_sum = school_data.budget.sum()

#Calculate the average math score
avg_math = school_data_complete.math_score.mean()

#Calculate the average reading score
avg_reading = school_data_complete.reading_score.mean()

#Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2
avg_overall = ((school_data_complete.reading_score.mean() + school_data_complete.math_score.mean()) / 2)

#Calculate the percentage of students with a passing math score (70 or greater)
passing_math = school_data_complete.loc[(school_data_complete.math_score >= 70), :]
pass_math_percent = passing_math.math_score.count() / school_data_complete.student_name.count()

#Calculate the percentage of students with a passing reading score (70 or greater)
passing_reading = school_data_complete.loc[(school_data_complete.reading_score >= 70), :]
pass_reading_percent = passing_reading.reading_score.count() / school_data_complete.student_name.count()

#Create summary table
summary = pd.DataFrame({"Tot_Schools" : [schools],
                        "Tot_Students" : [students],
                        "Tot_Budget" : [budget_sum],
                        "Avg_Math_Score" : [avg_math],
                        "Avg_Reading_Score" : [avg_reading],
                        "Avg_Overall_Score" : [avg_overall],
                        "Pass_Math_Percent" : [pass_math_percent],
                        "Pass_Read_Percent" : [pass_reading_percent]})

summary

,Tot_Schools,Tot_Students,Tot_Budget,Avg_Math_Score,Avg_Reading_Score,Avg_Overall_Score,Pass_Math_Percent,Pass_Read_Percent
0,15,39170,24649428,78.985371,81.87784,80.431606,0.749809,0.858055


In [3]:
#Group schools by name and type
grouped_schools = school_data_complete.groupby(["school_name","type"])

#Get sum, count, and mean datasets from the groupby object
sum_schools = pd.DataFrame(grouped_schools.sum().reset_index())
count_schools = pd.DataFrame(grouped_schools.count().reset_index())
avg_schools = pd.DataFrame(grouped_schools.mean().reset_index())

pass_math_per_school = passing_math.groupby(["school_name"])
pass_math_per_school = pass_math_per_school["Student ID"].count().reset_index()
#pass_math_per_school

pass_reading_per_school = passing_reading.groupby(["school_name"])
pass_reading_per_school = pass_reading_per_school["Student ID"].count().reset_index()
#pass_reading_per_school

#Use the .agg function to add in different components from the tables created above
aggregate = school_data_complete.groupby(["school_name","type"]).agg({'Student ID': 'count', 'budget': 'mean',
                                                'reading_score' : 'mean', 'math_score' : 'mean'}).reset_index()

aggregate["budget_per_student"] = aggregate["budget"] / aggregate["Student ID"]

aggregate = pd.merge(aggregate, pass_math_per_school, how="left", on=["school_name", "school_name"])
aggregate = pd.merge(aggregate, pass_reading_per_school, how="left", on=["school_name", "school_name"])

#Rename columns
aggregate = aggregate.rename(columns={"school_name" : "school_name",
                                       "type" : "school_type",
                                       "Student ID_x" : "tot_students",
                                       "budget" : "tot_budget",
                                       "reading_score" : "avg_reading_score",
                                       "math_score" : "avg_math_score",
                                       "Student ID_y" : "passed_math",
                                       "Student ID" : "passed_reading"})

#Perform calculations on the aggregate table to add the passing percentages
aggregate["passed_math_percent"] = aggregate["passed_math"] / aggregate["tot_students"]
aggregate["passed_reading_percent"] = aggregate["passed_reading"] / aggregate["tot_students"]
aggregate["overall_passing_percent"] = (aggregate["passed_math_percent"] + aggregate["passed_reading_percent"]) / 2

#Drop unwanted columns, reset index, rename columns
school_summary = aggregate.drop(["passed_math","passed_reading"], axis=1)
school_summary.reset_index(drop=True)
school_summary = school_summary.rename(columns={"school_name" : "School Name",
                                                "school_type" : "School Type",
                                                "tot_students" : "Total Students",
                                                "tot_budget" : "Total School Budget",
                                                "avg_math_score" : "Average Math Score",
                                                "avg_reading_score" : "Average Reading Score",
                                                "passed_math_percent" : "% Passing Math",
                                                "passed_reading_percent" : "% Passing Reading",
                                                "overall_passing_percent" : "Overall Passing Rate"})

school_summary.sort_values("budget_per_student")

,School Name,School Type,Total Students,Total School Budget,Average Reading Score,Average Math Score,budget_per_student,% Passing Math,% Passing Reading,Overall Passing Rate
13,Wilson High School,Charter,2283,1319574,83.989488,83.274201,578.0,0.938677,0.965396,0.952037
6,Holden High School,Charter,427,248087,83.814988,83.803279,581.0,0.925059,0.962529,0.943794
1,Cabrera High School,Charter,1858,1081356,83.975780,83.061895,582.0,0.941335,0.970398,0.955867
14,Wright High School,Charter,1800,1049400,83.955000,83.682222,583.0,0.933333,0.966111,0.949722
11,Shelton High School,Charter,1761,1056600,83.725724,83.359455,600.0,0.938671,0.958546,0.948609
9,Pena High School,Charter,962,585858,84.044699,83.839917,609.0,0.945946,0.959459,0.952703
4,Griffin High School,Charter,1468,917500,83.816757,83.351499,625.0,0.933924,0.971390,0.952657
0,Bailey High School,District,4976,3124928,81.033963,77.048432,628.0,0.666801,0.819333,0.743067
10,Rodriguez High School,District,3999,2547363,80.744686,76.842711,637.0,0.663666,0.802201,0.732933
12,Thomas High School,Charter,1635,1043130,83.848930,83.418349,638.0,0.932722,0.973089,0.952905


In [4]:
#Sort and display the top five schools in overall passing rate

top_passing = school_summary.sort_values(["Overall Passing Rate"], ascending=False).head(5)

top_passing = top_passing.set_index("School Name")

top_passing

,School Type,Total Students,Total School Budget,Average Reading Score,Average Math Score,budget_per_student,% Passing Math,% Passing Reading,Overall Passing Rate
School Name,,,,,,,,,
Cabrera High School,Charter,1858,1081356,83.975780,83.061895,582.0,0.941335,0.970398,0.955867
Thomas High School,Charter,1635,1043130,83.848930,83.418349,638.0,0.932722,0.973089,0.952905
Pena High School,Charter,962,585858,84.044699,83.839917,609.0,0.945946,0.959459,0.952703
Griffin High School,Charter,1468,917500,83.816757,83.351499,625.0,0.933924,0.971390,0.952657
Wilson High School,Charter,2283,1319574,83.989488,83.274201,578.0,0.938677,0.965396,0.952037


In [5]:
#Sort and display the five worst-performing schools

bottom_passing = school_summary.sort_values(["Overall Passing Rate"]).head(5)

bottom_passing = bottom_passing.set_index("School Name")

bottom_passing

,School Type,Total Students,Total School Budget,Average Reading Score,Average Math Score,budget_per_student,% Passing Math,% Passing Reading,Overall Passing Rate
School Name,,,,,,,,,
Rodriguez High School,District,3999,2547363,80.744686,76.842711,637.0,0.663666,0.802201,0.732933
Figueroa High School,District,2949,1884411,81.158020,76.711767,639.0,0.659885,0.807392,0.733639
Huang High School,District,2917,1910635,81.182722,76.629414,655.0,0.656839,0.813164,0.735002
Johnson High School,District,4761,3094650,80.966394,77.072464,650.0,0.660576,0.812224,0.736400
Ford High School,District,2739,1763916,80.746258,77.102592,644.0,0.683096,0.792990,0.738043


In [7]:
math = school_data_complete.groupby(["school_name","grade"])

math = math["math_score"].mean().reset_index()


#Filter the math dataframe to only inclue the grades that are needed to create new dataframes
math_nine = math.loc[(math["grade"] == "9th"), :].set_index("school_name")
math_ten = math.loc[(math["grade"] == "10th"), :].set_index("school_name")
math_eleven = math.loc[(math["grade"] == "11th"), :].set_index("school_name")
math_twelve = math.loc[(math["grade"] == "12th"), :].set_index("school_name")

#Join these together on the school to create a summary table
math_avgs = pd.merge(math_nine, math_ten, on=["school_name"], suffixes=('_nine', '_ten'))
math_avgs = pd.merge(math_avgs, math_eleven, on=["school_name"], suffixes=('_ten', '_eleven'))
math_avgs = pd.merge(math_avgs, math_twelve, on=["school_name"], suffixes=('_eleven', '_twelve'))

#Drop and rename
math_avgs = math_avgs.drop(["grade_nine","grade_ten","grade_eleven","grade_twelve"], axis = 1)
math_avgs = math_avgs.rename(columns = {"math_score_nine" : "9th",
                                        "math_score_ten" : "10th",
                                        "math_score_eleven" : "11th",
                                        "math_score_twelve" : "12th"})

math_avgs

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


In [8]:
reading = school_data_complete.groupby(["school_name","grade"])

reading = reading["reading_score"].mean().reset_index()


#Filter the reading dataframe to only inclue the grades that are needed to create new dataframes
reading_nine = reading.loc[(reading["grade"] == "9th"), :].set_index("school_name")
reading_ten = reading.loc[(reading["grade"] == "10th"), :].set_index("school_name")
reading_eleven = reading.loc[(reading["grade"] == "11th"), :].set_index("school_name")
reading_twelve = reading.loc[(reading["grade"] == "12th"), :].set_index("school_name")

#Join these together on the school to create a summary table
reading_avgs = pd.merge(reading_nine, reading_ten, on=["school_name"], suffixes=('_nine', '_ten'))
reading_avgs = pd.merge(reading_avgs, reading_eleven, on=["school_name"], suffixes=('_ten', '_eleven'))
reading_avgs = pd.merge(reading_avgs, reading_twelve, on=["school_name"], suffixes=('_eleven', '_twelve'))

#Drop and rename
reading_avgs = reading_avgs.drop(["grade_nine","grade_ten","grade_eleven","grade_twelve"], axis = 1)
reading_avgs = reading_avgs.rename(columns = {"reading_score_nine" : "9th",
                                              "reading_score_ten" : "10th",
                                              "reading_score_eleven" : "11th",
                                              "reading_score_twelve" : "12th"})

reading_avgs

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


In [9]:
#Create bins and group names
spend_bins = [0, 585, 615, 645, 675]
spend_names = ["<$585", "$585-615", "$615-645", "$645-675"]

#Add the bins to the dataframe
school_summary["Spend Category"] = pd.cut(school_summary["budget_per_student"], spend_bins, labels = spend_names)

#Group by the new bin category
school_grouped = school_summary.groupby(["Spend Category"])

school_spend_cats = school_grouped.mean().reset_index()

#Choose columns
school_spend_cats = school_spend_cats.loc[:, ["Spend Category", "Average Math Score",
                                              "Average Reading Score","% Passing Math",
                                              "% Passing Reading","Overall Passing Rate"]]

school_spend_cats = school_spend_cats.set_index("Spend Category")

school_spend_cats



,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Spend Category,,,,,
<$585,83.455399,83.933814,0.934601,0.966109,0.950355
$585-615,83.599686,83.885211,0.942309,0.959003,0.950656
$615-645,79.079225,81.891436,0.756682,0.861066,0.808874
$645-675,76.997210,81.027843,0.661648,0.811340,0.736494


In [10]:
#Create bins and group names
size_bins = [0, 1000, 2000, 5000]
size_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

#Add the bins to the dataframe
school_summary["School Size"] = pd.cut(school_summary["Total Students"], size_bins, labels = size_names)

#Group by the new bin category
school_size_grouped = school_summary.groupby(["School Size"])

school_size_cats = school_size_grouped.mean().reset_index()

#Choose columns
school_size_cats = school_size_cats.loc[:, ["School Size", "Average Math Score", "Average Reading Score",
                                            "% Passing Math", "% Passing Reading","Overall Passing Rate"]]

school_size_cats = school_size_cats.set_index("School Size")

school_size_cats
                                            
                                            

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Size,,,,,
Small (<1000),83.821598,83.929843,0.935502,0.960994,0.948248
Medium (1000-2000),83.374684,83.864438,0.935997,0.967907,0.951952
Large (2000-5000),77.746417,81.344493,0.699634,0.827666,0.763650


In [11]:
#Group by school type
school_type_grouped = school_summary.groupby(["School Type"])

school_type_cats = school_type_grouped.mean().reset_index()

#Choose columns
school_type_cats = school_type_cats.loc[:, ["School Type", "Average Math Score",
                                            "Average Reading Score","% Passing Math",
                                            "% Passing Reading","Overall Passing Rate"]]

school_type_cats = school_type_cats.set_index("School Type")

school_type_cats

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,0.936208,0.965865,0.951037
District,76.956733,80.966636,0.665485,0.807991,0.736738


## Analysis/Findings

There are two findings that stand out the most when looking over the results.  The first is that budget per student does not correlate with overall success of the students, in reading, math, or their overall passing rate.  Schools that have a great amount of funding or think that more money is the solution to fix their problems, may not always be correct and there are other factors that need to be looked at in order to make the students successful.

Another finding is that larger schools' students do not perform as well as the students at smaller schools.  While this is telling, missing data on the number of teachers at these schools is an important piece of missing information.  If the student/teacher ratio could be looked at, we would be able determine if the problem is the school size or the staff size.

Because of this data not being in this dataset, the other significant summary in this study was when looking at schools by school type.  Charter schools perform much better than District schools in every category, especially when looking at passing scores for math, reading, and overall.  Finding the common and uncommon traits between these two school types may help to bridge the gap.

As a side note, I created my calculations for the results to match up with the answers in the "PyCitySchools_starter.ipynb" file provided.  I don't fully understand why the last three summary tables (budget per student/school size/school type) would be taking averages of the averages from the School Summary table (from the third block of code).  I would think the best way to go about this would be taking the averages of all the students' scores at those schools, rather than the schools' average scores.  I don't think it made a huge difference in this dataset, but if one of the schools' overall passing rates was 20%, and there were 50 students who attended that school, it would have a large effect on the average of that and and successful school, let's say 95% with 3000 students attending (57.5% versus 93.8% for the weighted average).  This is just what I was thinking and how I would usually go about something like this, but I wanted to mention it to see if I was thinking of this in an incorrect way.